# Introdução


Vamos usar os conhecimentos da aula passada (Validação I) para encontrar o melhor valor de vizinhos mais próximos para um classificador ```KNeighborsClassifier```. Lembre que mostramos algumas formas de validar modelos de machine learning, que nada mais é um processo de estimar o erro/performance que um modelo terá em produção. A partir dos resultados, podemos checar se o modelo sofre de overfitting ou underfitting. Vamos utilizar essa mesma técnica para identificar de que formas podemos melhorar nosso modelo.



# Setup

In [54]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,learning_curve,KFold, StratifiedKFold, LeaveOneOut, cross_validate, validation_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, make_scorer
import matplotlib.pyplot as plt

In [62]:
#dados
df_churn = pd.read_csv('churn.csv')

#  K-vizinhos (Default)

Antes de tudo, vamos preparar nossos Folds de validação cruzada. Vamos preparar a validação KFold, KFold-Estratificada e Leave One Out.

## KFold

## KFold Estratificada

## Leave One Out

In [80]:
# correção para log loss 
def loss_scorer(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None):
    return -log_loss(y_true, y_pred, eps, normalize, sample_weight, labels=__LABELS__)
global __LABELS__
__LABELS__ = list(set(df_churn['Churn']))
scorer_loss = make_scorer(loss_scorer, greater_is_better=True, needs_proba=True)

# Otimização dos K-Vizinhos

vamos criar uma lista de parâmetros com vários valores de vizinhos mais próximos e tentar obter o melhor, baseando-se na ```validation_curve```.

Vamos criar uma função auxiliar que irá realizar os cálculos da ```validation_curve``` e ao mesmo tempo plotar o gráfico.

## KFold Estratificado

# Curva de Aprendizado

Já falamos um pouco sobre trade-off viés-variância. Temos sempre que encontrar um balanço entre os dois, buscando um modelo que não é nem tão complexo nem tão simples pra resolver nosso problema. Se um modelo é  muito simpoles, não vai conseguir ser flexível o suficiente para capturar os padrões dos dados, ao passo que um modelo muito complexo, que é muito flexível, vai acabar se ajustando demais aos dados de treino e é muito sensível a eles, de forma que ele acaba decorando o ruído e não generalizando bem para dados de produção.


<img src="https://storybydata.com/wp-content/uploads/2019/02/1_9hPX9pAO3jqLrzt0IE3JzA.png" width=600>

O que você pode fazer, caso seu modelo não fique bom depois da modelagem? Há algumas coisas que é possível fazer, quando isso ocorre:
- Coletar mais dados
- Diminuir o conjunto de features. Isso mesmo, diminuir o conjunto de features vai reduzir a complexidade de um modelo, diminuindo a propensão ao overfitting.
- Aumentar o conjunto de features. Poder ser pela coleta de mais colunas ou então pela criação, por meio do processo de feature engineering que vimos. Isso aumentará a complexidade do modelo, o que reduzirá underfitting.
- Redução de regularização. A redução de regularização varia de modelo para modelo. No SVM, por exemplo, a regularização é o parâmetro C. Mas há outros parâmetros em outros modelos, que fazem esse papel.Isso aumentará a complexidade do modelo e reduzirá a propensão a underfitting.
- Aumento de regularização. O aumento de regularização, por sua vez, nos traz um redução de complexidade, o que ajudará em casos de overfitting.



Maravilha, agora sabemos como agir diante de um modelo ruim. Mas a questão que fica é:
Como sabemos qual desses métodos tentar primeiro?. 
A resposta é simples: Depende. Vai depender se o seu modelo tá sofrendo de alto viés ou alta variância.
E como podemos detectar alto viés o alta variância? Isso mesmo! Curvas de aprendizado.

Curvas de aprendizado mostram a relação entre o tamanho do conjunto de treinamento e a métrica de avaliação do modelo no conjunto de validação. 

A ideia principal é que se o algoritmo estiver aprendendo, quanto mais dados fornecemos melhor ele fica.

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/models.png" width=500>

Com mais dados de treino, mais ajustado vai ficando o modelo

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/learning_curves.png" width=600>

Tipos de Curva:

- Alto Bias e Baixa Variância

A curva abaixo sofre de alto viés e baixa variância. Note que quando aumentamos o conjunto de treino, praticamente nada muda. Ou seja, o modelo não está aprendendo bem. Além disso, o erro é alto tanto no treino quanto no teste.

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/Learning_curves_12_1.png" width=400>

Esse gráfico nos mostra algo importante: Não adianta coletar mais dados, porque não há nenhuma expectativa de melhoria com isso. Em vez disso, poderíamos tentar mudar o modelo (pegar um modelo mais flexível que consiga entender padrões mais complexos), ou até mesmo adicionar features. Note que adicionar features é diferente de adicionar mais linhas! No nosso caso, adicionar features aumentaria a complexidade do modelo e é disso que precisamos.

EM suma, um erro alto de validação indica problema com viés. Para saber se o viés está alto ou baixo, olhamos a curva de treino. Quando o erro do treino também é alto, temos um caso de viés alto. Quando o erro do treino é baixo, temos um caso de viés baixo. 

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/low_high_bias.png" width=600>

Para analisar a variância, por sua vez, podemos ver o gap entre as duas curvas. um Gap muito grande, indica que as performances estão bem diferentes, o que indica alta variância.

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/low_high_var.png" width=600>

Pelo formato de nossa curva, podemos concluir que se trata de um caso de Alto viés e Baixa variância. O erro é alto no treino e na validação e as curvas são próximas.

- ALta Variância e Baixo Bias

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/Learning_curves_15_0.png" width=600>

Nesse caso, mudamos o modelo para um random forest. O random forest é um modelo mais flexível e, portanto, consegue ajustar melhor os dados. Note que o erro de treino decresceu consideravelmente e o erro de validação também decresceu. Contudo, aumentamos o gap entre as curvas. De forma geral, dizemos que o Viés baixou (decorrente de um modelo mais flexível) e a variância aumentou.

Nesse caso, podemos:
- Coletar mais dados
- Aumentar a regularização do algoritmo usado
- Reduzir o número de features. Feature Selection Matters!!!

- Baixa Variâcia e Baixo Viés

Quando conseguimos controlar viés e variância bem, encontramos o erro irredutível. Na prática é muito difícil de alcançá-lo. Principalmente, pois não sabemos o seu valor!. Na vida real, o que vai limitar sua busca por um erro irredutível é o tempo e os recursos disponíveis!.

<img src="https://www.dataquest.io/wp-content/uploads/2019/01/irr_error.png" width=600>

Para mais info sobre learning curves: 
    - https://www.ritchieng.com/machinelearning-learning-curve/
    - https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

Vamos checar a curva de aprendizado do nosso modelo?